## Bayesian model comparison

# Intuition

**FIGURE PLACEHOLDER:** ![Bayesian model comparison](image_placeholder)

# Notation


### Compare $M_1$ and $M_2$ based on data $D$
When comparing two models $M_1$ and $M_2$, we calculate the **posterior odds** for each model. The **posterior odds** represent the ratio of the posterior probabilities for the two models. This can be written as:

$$
\frac{P(M_1|D)}{P(M_2|D)} = \frac{P(D|M_1)P(M_1)}{P(D|M_2)P(M_2)}
$$

Where:
- $P(M_1|D)$ is the **posterior** probability of model $M_1$ given the data $D$.
- $P(M_2|D)$ is the **posterior** probability of model $M_2$ given the data $D$.
- $P(D|M_1)$ and $P(D|M_2)$ are the **likelihoods** of the data under models $M_1$ and $M_2$, respectively.
- $P(M_1)$ and $P(M_2)$ are the **prior probabilities** of models $M_1$ and $M_2$, respectively.

This ratio simplifies to:

$$
\text{Posterior Odds} = \text{Prior Odds} \times \text{Likelihood Ratio (LR)}
$$

Where:
- **Prior Odds** = $\frac{P(M_1)}{P(M_2)}$ is the ratio of prior probabilities of the models.
- **Likelihood Ratio (LR)** = $\frac{P(D|M_1)}{P(D|M_2)}$ is the ratio of the likelihoods of the data under each model.

Thus, the **posterior odds** can be updated by multiplying the **prior odds** by the **likelihood ratio**.


To compare two models $M_1$ and $M_2$ in the multivariate case, we calculate the **posterior odds** for each model, just as we did in the univariate case. This allows us to update our belief about which model is more likely, given the observed data.

The **posterior odds** are given by the ratio of the posterior probabilities of each model, which is the Bayes factor:

$$
\text{Posterior Odds} = \frac{P(M_1 | D)}{P(M_2 | D)}
$$

Using Bayes' theorem, this can be expressed as:

$$
\text{Posterior Odds} = \frac{P(D | M_1) P(M_1)}{P(D | M_2) P(M_2)}
$$

Where:
- $P(M_1 | D)$ and $P(M_2 | D)$ are the posterior probabilities of models $M_1$ and $M_2$ given the data $D$.
- $P(D | M_1)$ and $P(D | M_2)$ are the **likelihoods** of the observed data given the models $M_1$ and $M_2$.
- $P(M_1)$ and $P(M_2)$ are the **prior probabilities** of the models.

This ratio, known as the **Bayes Factor**, quantifies the strength of evidence in favor of one model over the other:

$$
\text{Bayes Factor} = \frac{P(D | M_1)}{P(D | M_2)}
$$

If the Bayes Factor is greater than 1, it suggests that $M_1$ is more likely given the data, while a Bayes Factor less than 1 suggests that $M_2$ is more likely. The larger the Bayes Factor, the stronger the evidence in favor of one model. A Bayes Factor close to 1 indicates that the models are equally supported by the data.

In practice, the posterior probabilities for each model are computed by integrating the likelihood over the parameter space of the model and multiplying it by the prior probability. This allows us to calculate the **posterior distribution** for each model, which captures the uncertainty about the model’s parameters after observing the data.

Therefore, model comparison in the Bayesian framework is not just about selecting a single best model but quantifying and comparing the plausibility of different models based on how well they fit the observed data.


# Example

## Model comparison
TODO
